#### Step 1 : Clean and normalise data

**What doesn't interest us:**
- appid, price (too much missing data), dlc_count (not significative), metacritic_score (too much missing data), reviews ? (too much missing data)

**Important columns:**

- release_date -> essential for year progression and game filtering
- recommendations -> good for game popularity
- genres -> main column to take into account
- tags -> secondary main column to take into account
- estimated_owners -> good to get an idea of the game's popularity
- 

**Columns for prediction:**
- positive, negative -> essential for predictions
- average_playtime_forever, average_playtime_2weeks -> some missing data, but peak ccu & estimated owners can help
- median_playtime_forever, median_playtime_2weeks -> same as average
- peak ccu -> game popularity
- pct_pos_total, pct_pos_recent, num_reviews_total, num_reviews_recent -> essential to get an idea of evolution in the future

**Might be useful, or serve as guidelines:**
- name
- categories

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("dataset\steam_dataset_2025.csv", encoding="cp1252", sep=';')

df.head()

,appid,name,release_date,price,dlc_count,reviews,metacritic_score,recommendations,categories,genres,...,average_playtime_forever,average_playtime_2weeks,median_playtime_forever,median_playtime_2weeks,peak_ccu,tags,pct_pos_total,num_reviews_total,pct_pos_recent,num_reviews_recent
0,730,Counter-Strike 2,21/08/2012,0.00,1,NaN,0,4401572,"['Multi-player', 'Cross-Platform Multiplayer',...","['Action', 'Free To Play']",...,33189,879,5174,350,1212356,"{'FPS': 90857, 'Shooter': 65397, 'Multiplayer'...",86,8632939,82,96473
1,578080,PUBG: BATTLEGROUNDS,21/12/2017,0.00,0,NaN,0,1732007,"['Multi-player', 'PvP', 'Online PvP', 'Stats',...","['Action', 'Adventure', 'Massively Multiplayer...",...,0,0,0,0,616738,"{'Survival': 14838, 'Shooter': 12727, 'Battle ...",59,2513842,68,16720
2,570,Dota 2,09/07/2013,0.00,2,“A modern multiplayer masterpiece.” 9.5/10 – D...,90,14337,"['Multi-player', 'Co-op', 'Steam Trading Cards...","['Action', 'Strategy', 'Free To Play']",...,43031,1536,898,892,555977,"{'Free to Play': 59933, 'MOBA': 20158, 'Multip...",81,2452595,80,29366
3,271590,Grand Theft Auto V Legacy,13/04/2015,0.00,0,NaN,96,1803063,"['Single-player', 'Multi-player', 'PvP', 'Onli...","['Action', 'Adventure']",...,19323,771,7101,74,117698,"{'Open World': 32644, 'Action': 23539, 'Multip...",87,1803832,92,17517
4,359550,Tom Clancy's Rainbow Six® Siege,01/12/2015,3.99,9,NaN,0,1165929,"['Single-player', 'Multi-player', 'PvP', 'Onli...",['Action'],...,14204,682,2434,306,89916,"{'FPS': 9831, 'PvP': 9162, 'e-sports': 9072, '...",84,1168020,76,12608


In [ ]:
#Checking info on the dataset

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89618 entries, 0 to 89617
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   appid                     89618 non-null  int64  
 1   name                      89618 non-null  object 
 2   release_date              89618 non-null  object 
 3   price                     89618 non-null  float64
 4   dlc_count                 89618 non-null  int64  
 5   reviews                   10401 non-null  object 
 6   metacritic_score          89618 non-null  int64  
 7   recommendations           89618 non-null  int64  
 8   categories                89618 non-null  object 
 9   genres                    89618 non-null  object 
 10  positive                  89618 non-null  int64  
 11  negative                  89618 non-null  int64  
 12  estimated_owners          89618 non-null  object 
 13  average_playtime_forever  89618 non-null  int64  
 14  averag

A lot of reviews are missing, and a lot of elements are described as 'non-null', but are in fact '0', which can be described as null.

For exampe, Metacritic doesn't usually give 0 score, but there are many 0s in 'metacritic_score'.  It isn't normal that PUBG or Country Strike 2, which are very popular shooting games, have 0. Thus, the information mustn't have been found, and 0 was instead given to indicate a missing value. That is why 0 mustn't be considered bad grades, but rather, skippable grades. Values between 1 and 40 should be considered bad grades though. 

While metacritic_score could have been an interesting category for predictions, there are too many 0s, as indicated in the following cell.

In [ ]:
df['metacritic_score'].value_counts()

#80k+ '0'. We're better off deleting this category, instead of finding the scores. The positive numbers and recommendations columns are enough.

metacritic_score
0     86071
80      195
81      169
78      163
76      159
      ...  
23        1
20        1
29        1
35        1
30        1
Name: count, Length: 69, dtype: int64

Moreoever, we can notice 2 wrong types of data:

- release_date -> object -> int64
- estimated_owners -> object -> int64

**Things to edit:**
- remove unnecessary columns OK
- release_date -> DD/MM/YY --> Year only OK
- edit wrong data types (handle 'estimated_owners' aspect) OK
- remove numbers to the tags

In [10]:
#Removing unnecessary columns

df = df.drop(["appid", "price", "dlc_count", "metacritic_score"], axis=1)

df.head()

,name,release_date,reviews,recommendations,categories,genres,positive,negative,estimated_owners,average_playtime_forever,average_playtime_2weeks,median_playtime_forever,median_playtime_2weeks,peak_ccu,tags,pct_pos_total,num_reviews_total,pct_pos_recent,num_reviews_recent
0,Counter-Strike 2,21/08/2012,NaN,4401572,"['Multi-player', 'Cross-Platform Multiplayer',...","['Action', 'Free To Play']",7480813,1135108,100000000 - 200000000,33189,879,5174,350,1212356,"{'FPS': 90857, 'Shooter': 65397, 'Multiplayer'...",86,8632939,82,96473
1,PUBG: BATTLEGROUNDS,21/12/2017,NaN,1732007,"['Multi-player', 'PvP', 'Online PvP', 'Stats',...","['Action', 'Adventure', 'Massively Multiplayer...",1487960,1024436,50000000 - 100000000,0,0,0,0,616738,"{'Survival': 14838, 'Shooter': 12727, 'Battle ...",59,2513842,68,16720
2,Dota 2,09/07/2013,“A modern multiplayer masterpiece.” 9.5/10 – D...,14337,"['Multi-player', 'Co-op', 'Steam Trading Cards...","['Action', 'Strategy', 'Free To Play']",1998462,451338,200000000 - 500000000,43031,1536,898,892,555977,"{'Free to Play': 59933, 'MOBA': 20158, 'Multip...",81,2452595,80,29366
3,Grand Theft Auto V Legacy,13/04/2015,NaN,1803063,"['Single-player', 'Multi-player', 'PvP', 'Onli...","['Action', 'Adventure']",1719950,250012,50000000 - 100000000,19323,771,7101,74,117698,"{'Open World': 32644, 'Action': 23539, 'Multip...",87,1803832,92,17517
4,Tom Clancy's Rainbow Six® Siege,01/12/2015,NaN,1165929,"['Single-player', 'Multi-player', 'PvP', 'Onli...",['Action'],1152763,218446,20000000 - 50000000,14204,682,2434,306,89916,"{'FPS': 9831, 'PvP': 9162, 'e-sports': 9072, '...",84,1168020,76,12608


In [11]:
#Remodelling release_date

df['release_date'] = df['release_date'].str.split(pat="/", expand=True)[2]

df.head()

,name,release_date,reviews,recommendations,categories,genres,positive,negative,estimated_owners,average_playtime_forever,average_playtime_2weeks,median_playtime_forever,median_playtime_2weeks,peak_ccu,tags,pct_pos_total,num_reviews_total,pct_pos_recent,num_reviews_recent
0,Counter-Strike 2,2012,NaN,4401572,"['Multi-player', 'Cross-Platform Multiplayer',...","['Action', 'Free To Play']",7480813,1135108,100000000 - 200000000,33189,879,5174,350,1212356,"{'FPS': 90857, 'Shooter': 65397, 'Multiplayer'...",86,8632939,82,96473
1,PUBG: BATTLEGROUNDS,2017,NaN,1732007,"['Multi-player', 'PvP', 'Online PvP', 'Stats',...","['Action', 'Adventure', 'Massively Multiplayer...",1487960,1024436,50000000 - 100000000,0,0,0,0,616738,"{'Survival': 14838, 'Shooter': 12727, 'Battle ...",59,2513842,68,16720
2,Dota 2,2013,“A modern multiplayer masterpiece.” 9.5/10 – D...,14337,"['Multi-player', 'Co-op', 'Steam Trading Cards...","['Action', 'Strategy', 'Free To Play']",1998462,451338,200000000 - 500000000,43031,1536,898,892,555977,"{'Free to Play': 59933, 'MOBA': 20158, 'Multip...",81,2452595,80,29366
3,Grand Theft Auto V Legacy,2015,NaN,1803063,"['Single-player', 'Multi-player', 'PvP', 'Onli...","['Action', 'Adventure']",1719950,250012,50000000 - 100000000,19323,771,7101,74,117698,"{'Open World': 32644, 'Action': 23539, 'Multip...",87,1803832,92,17517
4,Tom Clancy's Rainbow Six® Siege,2015,NaN,1165929,"['Single-player', 'Multi-player', 'PvP', 'Onli...",['Action'],1152763,218446,20000000 - 50000000,14204,682,2434,306,89916,"{'FPS': 9831, 'PvP': 9162, 'e-sports': 9072, '...",84,1168020,76,12608


In [15]:
# Modify 'estimated_owners' aspect : split the column into two separate columns 'owners_low' and 'owners_high' + fix to the right datatype

df[['owners_low', 'owners_high']] = df["estimated_owners"].str.split(" - ", expand=True).astype(int)

df = df.drop("estimated_owners", axis=1)

df.head()

,name,release_date,reviews,recommendations,categories,genres,positive,negative,average_playtime_forever,average_playtime_2weeks,median_playtime_forever,median_playtime_2weeks,peak_ccu,tags,pct_pos_total,num_reviews_total,pct_pos_recent,num_reviews_recent,owners_low,owners_high
0,Counter-Strike 2,2012,NaN,4401572,"['Multi-player', 'Cross-Platform Multiplayer',...","['Action', 'Free To Play']",7480813,1135108,33189,879,5174,350,1212356,"{'FPS': 90857, 'Shooter': 65397, 'Multiplayer'...",86,8632939,82,96473,100000000,200000000
1,PUBG: BATTLEGROUNDS,2017,NaN,1732007,"['Multi-player', 'PvP', 'Online PvP', 'Stats',...","['Action', 'Adventure', 'Massively Multiplayer...",1487960,1024436,0,0,0,0,616738,"{'Survival': 14838, 'Shooter': 12727, 'Battle ...",59,2513842,68,16720,50000000,100000000
2,Dota 2,2013,“A modern multiplayer masterpiece.” 9.5/10 – D...,14337,"['Multi-player', 'Co-op', 'Steam Trading Cards...","['Action', 'Strategy', 'Free To Play']",1998462,451338,43031,1536,898,892,555977,"{'Free to Play': 59933, 'MOBA': 20158, 'Multip...",81,2452595,80,29366,200000000,500000000
3,Grand Theft Auto V Legacy,2015,NaN,1803063,"['Single-player', 'Multi-player', 'PvP', 'Onli...","['Action', 'Adventure']",1719950,250012,19323,771,7101,74,117698,"{'Open World': 32644, 'Action': 23539, 'Multip...",87,1803832,92,17517,50000000,100000000
4,Tom Clancy's Rainbow Six® Siege,2015,NaN,1165929,"['Single-player', 'Multi-player', 'PvP', 'Onli...",['Action'],1152763,218446,14204,682,2434,306,89916,"{'FPS': 9831, 'PvP': 9162, 'e-sports': 9072, '...",84,1168020,76,12608,20000000,50000000


In [ ]:
#Rename 'release_date' to 'release_year' for less confusion

df.rename(columns={"release_date":"release_year"}, inplace=True)

In [20]:
#Fix release_year datatype

df['release_year'] = df["release_year"].astype('int64')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89618 entries, 0 to 89617
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   name                      89618 non-null  object
 1   release_year              89618 non-null  int64 
 2   reviews                   10401 non-null  object
 3   recommendations           89618 non-null  int64 
 4   categories                89618 non-null  object
 5   genres                    89618 non-null  object
 6   positive                  89618 non-null  int64 
 7   negative                  89618 non-null  int64 
 8   average_playtime_forever  89618 non-null  int64 
 9   average_playtime_2weeks   89618 non-null  int64 
 10  median_playtime_forever   89618 non-null  int64 
 11  median_playtime_2weeks    89618 non-null  int64 
 12  peak_ccu                  89618 non-null  int64 
 13  tags                      89618 non-null  object
 14  pct_pos_total         

In [21]:
#Remove numbers to 'tags'

#The tags are presented as a dictionary aspect : {'FPS': 90857, ...}. The number corresponds to the amount of people who tagged the game as X genre.
#For example, 90857 people tagged Counter-Strike 2 as a FPS game. But this doesn't help us in identifying whether the game is popular. The amount of 
#tags could, but there are better columns that can help us with that. For now, we can remove the numbers, and keep the tags only.

# Step 1: parse the column values as dictionaries, and not strings. Split() won't help us because of the amount of different potential separators.
import ast

df['tags_dict'] = df["tags"].apply(ast.literal_eval)

df["tags"] = df["tags_dict"].apply(lambda d: list(d.keys()))

AttributeError: 'list' object has no attribute 'keys'